In [1]:
from mimetypes import init
from turtle import color
import Toolkit as tk
import ToyModel as tm
import torch
import torch.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle
import time
import ray
import os
import seaborn  as sns
import matplotlib.pyplot as plt
import warnings
import json
import rich
import multiprocessing as mp
import cobra

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-17


In [2]:
NUM_CORES = 8
warnings.filterwarnings("ignore") 

In [6]:
agent1=tk.Agent("agent1",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=10,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1' ,'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agents=[agent1]


Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmp7thmvi7a.lp
Reading time = 0.00 seconds
: 7 rows, 18 columns, 44 nonzeros


In [7]:
env=tk.Environment(name="Toy-Exoenzyme-Single-agents",
                    agents=agents,
                    dilution_rate=0.0001,
                    initial_condition={"Glc":100,"agent1":0.1,"Starch":10},
                    inlet_conditions={"Starch":10},
                    extracellular_reactions=[{"reaction":{
                    "Glc":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Glc","Amylase"))}],
                    max_c={'Glc':100,
                           'agent1':10,  
                           'Starch':10,
                           },
                           dt=0.1,
                           episode_time=100,
                           number_of_batches=5000,
                           episodes_per_batch=int(NUM_CORES/2),
                           )

The following species are not in the community: ['Starch']


Environment Toy-Exoenzyme-Single-agents created successfully!.


In [10]:
env.rewards={agent.name:[] for agent in env.agents}

if not os.path.exists(f"Results/{env.name}"):
	os.makedirs(f"Results/{env.name}")

for agent in env.agents:
	agent.model.solver="glpk"

for batch in range(env.number_of_batches):

	batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
	for agent in env.agents:
		V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
		A_k = batch_rtgs[agent.name] - V.detach()   
		A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
		if batch==0:
			rich.print("[bold yellow] Hold on, bringing the creitc network to range...[/bold yellow]")
			err=21
			while err>20:
				V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step() 
				err=critic_loss.item()
			rich.print("[bold green] Done![/bold green]")
		else: 
			
			for _ in range(agent.grad_updates):                                                      
				V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
				surr1 = ratios * A_k.detach()
				surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
				actor_loss = (-torch.min(surr1, surr2)).mean()
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_policy_.zero_grad()
				actor_loss.backward(retain_graph=False)
				agent.optimizer_policy_.step()
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step()                                                            
	
		if batch%500==0:
		
			with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
				pickle.dump(env, f)
			with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_obs,f)		
			with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_acts,f)		
          		
		print(f"Batch {batch} finished:")
		for agent in env.agents:
			print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")		


KeyboardInterrupt: 

In [11]:
agent1=tk.Agent("agent1",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=10,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2' ,'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agent2=tk.Agent("agent2",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=10,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agents=[agent1,agent2]

env=tk.Environment(name="Toy-Exoenzyme-Two-agents",
                    agents=agents,
                    dilution_rate=0.0001,
                    initial_condition={"Glc":100,"agent1":0.1,"agent2":0.1,"Starch":10},
                    inlet_conditions={"Starch":10},
                    extracellular_reactions=[{"reaction":{
                    "Glc":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Glc","Amylase"))}],
                    max_c={'Glc':100,
                           'agent1':10,  
                           'Starch':10,
                           },
                           dt=0.1,
                           episode_time=100,
                           number_of_batches=5000,
                           episodes_per_batch=int(NUM_CORES/2),
                           )



Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpalrbsvhd.lp
Reading time = 0.00 seconds
: 7 rows, 18 columns, 44 nonzeros
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmp566lcqnu.lp
Reading time = 0.00 seconds
: 7 rows, 18 columns, 44 nonzeros


The following species are not in the community: ['Starch']


Environment Toy-Exoenzyme-Two-agents created successfully!.


In [13]:
env.rewards={agent.name:[] for agent in env.agents}

if not os.path.exists(f"Results/{env.name}"):
	os.makedirs(f"Results/{env.name}")

for agent in env.agents:
	agent.model.solver="glpk"

for batch in range(env.number_of_batches):

	batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
	for agent in env.agents:
		V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
		A_k = batch_rtgs[agent.name] - V.detach()   
		A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
		if batch==0:
			rich.print("[bold yellow] Hold on, bringing the creitc network to range...[/bold yellow]")
			err=21
			while err>20:
				V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step() 
				err=critic_loss.item()
			rich.print("[bold green] Done![/bold green]")
		else: 
			
			for _ in range(agent.grad_updates):                                                      
				V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
				surr1 = ratios * A_k.detach()
				surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
				actor_loss = (-torch.min(surr1, surr2)).mean()
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_policy_.zero_grad()
				actor_loss.backward(retain_graph=False)
				agent.optimizer_policy_.step()
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step()                                                            
	
		if batch%500==0:
		
			with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
				pickle.dump(env, f)
			with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_obs,f)		
			with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_acts,f)		
          		
		print(f"Batch {batch} finished:")
		for agent in env.agents:
			print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")	

 Hold on, bringing the creitc network to range...

 Done!

Batch 0 finished:
agent1 return is:  -834.9255180536776
agent2 return is:  -815.9404837891703


 Hold on, bringing the creitc network to range...

 Done!

Batch 0 finished:
agent1 return is:  -834.9255180536776
agent2 return is:  -815.9404837891703
Batch 1 finished:
agent1 return is:  -841.8440040666383
agent2 return is:  -826.1826531004805
Batch 1 finished:
agent1 return is:  -841.8440040666383
agent2 return is:  -826.1826531004805
Batch 2 finished:
agent1 return is:  -835.6765195038785
agent2 return is:  -820.6359854544567
Batch 2 finished:
agent1 return is:  -835.6765195038785
agent2 return is:  -820.6359854544567
Batch 3 finished:
agent1 return is:  -838.8587132565848
agent2 return is:  -824.0484213947282
Batch 3 finished:
agent1 return is:  -838.8587132565848
agent2 return is:  -824.0484213947282
Batch 4 finished:
agent1 return is:  -841.3364360564342
agent2 return is:  -832.9258086808351
Batch 4 finished:
agent1 return is:  -841.3364360564342
agent2 return is:  -832.9258086808351


KeyboardInterrupt: 

In [6]:
agent1=tk.Agent("agent1",
				model=tm.Toy_Model_NE_Aux_1,
				actor_network=tk.NN,
				critic_network=tk.NN,
				clip=0.1,
				lr_actor=0.0001,
				lr_critic=0.001,
				grad_updates=10,
				optimizer_actor=torch.optim.Adam,
				optimizer_critic=torch.optim.Adam,       
				observables=['agent1','agent2','S',"A","B"],
				actions=['A_e','B_e'],
				gamma=1,
				)
agent2=tk.Agent("agent2",
				model=tm.Toy_Model_NE_Aux_2,
				actor_network=tk.NN,
				critic_network=tk.NN,
				clip=0.1,
				lr_actor=0.0001,
				lr_critic=0.001,
				grad_updates=10,
				optimizer_actor=torch.optim.Adam,
				optimizer_critic=torch.optim.Adam,       
				observables=['agent1','agent2','S',"A","B"],
				actions=['A_e','B_e'],
				gamma=1)
agents=[agent1,agent2]

env=tk.Environment(name="Toy-NECOM-Auxotrophs",
 					agents=agents,
 					dilution_rate=0.0001,
 					extracellular_reactions=[],
 					initial_condition={"S":100,"agent1":0.1,"agent2":0.1},
 					inlet_conditions={"S":100},
 					max_c={'S':100,
 						   'agent1':10,  
 						   'agent2':10,
 						   'A':10,
 						   'B':10,},
 							dt=0.1,
 							episode_time=100,
 							number_of_batches=5000,
 							episodes_per_batch=NUM_CORES,)

env.rewards={agent.name:[] for agent in env.agents}
if not os.path.exists(f"Results/{env.name}"):
	os.makedirs(f"Results/{env.name}")
for agent in env.agents:
	agent.model.solver="glpk"
for batch in range(env.number_of_batches):
	batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
	for agent in env.agents:
		V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
		A_k = batch_rtgs[agent.name] - V.detach()   
		A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
		if batch==0:
			rich.print("[bold yellow] Hold on, bringing the creitc network to range...[/bold yellow]")
			err=51
			while err>50:
				V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step() 
				err=critic_loss.item()
			rich.print("[bold green] Done![/bold green]")
		else: 			
			for _ in range(agent.grad_updates):                                                      
				V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
				surr1 = ratios * A_k.detach()
				surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
				actor_loss = (-torch.min(surr1, surr2)).mean()
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_policy_.zero_grad()
				actor_loss.backward(retain_graph=False)
				agent.optimizer_policy_.step()
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step()                                                            	
	if batch%500==0:		
		with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
			pickle.dump(env, f)
		with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
			pickle.dump(batch_obs,f)		
		with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
			pickle.dump(batch_acts,f)		
    		
	print(f"Batch {batch} finished:")
	for agent in env.agents:
		print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")		

Environment Toy-NECOM-Auxotrophs created successfully!.


 Hold on, bringing the creitc network to range...

 Done!

 Hold on, bringing the creitc network to range...

 Done!

Batch 0 finished:
agent1 return is:  -693.263211649023
agent2 return is:  -494.3485401777354
Batch 1 finished:
agent1 return is:  -689.7608206478573
agent2 return is:  -489.10863895799946
Batch 2 finished:
agent1 return is:  -686.385095640583
agent2 return is:  -500.47163949786096
Batch 3 finished:
agent1 return is:  -697.5145220951706
agent2 return is:  -505.0906397532352
Batch 4 finished:
agent1 return is:  -701.8949813944643
agent2 return is:  -510.2214796378634
Batch 5 finished:
agent1 return is:  -688.8917124463186
agent2 return is:  -506.8473380565723
Batch 6 finished:
agent1 return is:  -696.5160124816357
agent2 return is:  -501.72032076966195
Batch 7 finished:
agent1 return is:  -680.2570645901875
agent2 return is:  -490.71814468345343
Batch 8 finished:
agent1 return is:  -684.3835380369874
agent2 return is:  -464.32930403313355
Batch 9 finished:
agent1 return is:  -688.5053279676099
agent2 return is:  -459.6008065688017
Batch 10 finished:
agent1 return is:  -691.6378828485514


KeyboardInterrupt: 

In [3]:
agent1=tk.Agent("agent1",
				model=tm.Toy_Model_NE_Mut_1,
				actor_network=tk.NN,
				critic_network=tk.NN,
				clip=0.1,
				lr_actor=0.0001,
				lr_critic=0.001,
				grad_updates=10,
				optimizer_actor=torch.optim.Adam,
				optimizer_critic=torch.optim.Adam,       
				observables=['agent1','agent2','S',"A","B"],
				actions=['A_e','B_e'],
				gamma=1,
				)
agent2=tk.Agent("agent2",
				model=tm.Toy_Model_NE_Mut_2,
				actor_network=tk.NN,
				critic_network=tk.NN,
				clip=0.1,
				lr_actor=0.0001,
				lr_critic=0.001,
				grad_updates=10,
				optimizer_actor=torch.optim.Adam,
				optimizer_critic=torch.optim.Adam,       
				observables=['agent1','agent2','S',"A","B"],
				actions=['A_e','B_e'],
				gamma=1)
agents=[agent1,agent2]

env=tk.Environment(name="Toy-NECOM-Facultative",
 					agents=agents,
 					dilution_rate=0.0001,
 					extracellular_reactions=[],
 					initial_condition={"S":100,"agent1":0.1,"agent2":0.1,"A":10,"B":10},
 					inlet_conditions={"S":100},
 					max_c={'S':100,
 						   'agent1':10,  
 						   'agent2':10,
 						   'A':10,
 						   'B':10,},
 							dt=0.1,
 							episode_time=100,
 							number_of_batches=5000,
 							episodes_per_batch=int(NUM_CORES/2),)

env.rewards={agent.name:[] for agent in env.agents}
if not os.path.exists(f"Results/{env.name}"):
	os.makedirs(f"Results/{env.name}")
for agent in env.agents:
	agent.model.solver="glpk"
for batch in range(env.number_of_batches):
	batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
	for agent in env.agents:
		V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
		A_k = batch_rtgs[agent.name] - V.detach()   
		A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
		if batch==0:
			rich.print("[bold yellow] Hold on, bringing the creitc network to range...[/bold yellow]")
			err=21
			while err>20:
				V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step() 
				err=critic_loss.item()
			rich.print("[bold green] Done![/bold green]")
		else: 			
			for _ in range(agent.grad_updates):                                                      
				V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
				surr1 = ratios * A_k.detach()
				surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
				actor_loss = (-torch.min(surr1, surr2)).mean()
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_policy_.zero_grad()
				actor_loss.backward(retain_graph=False)
				agent.optimizer_policy_.step()
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step()                                                            	
		if batch%500==0:		
			with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
				pickle.dump(env, f)
			with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_obs,f)		
			with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
				pickle.dump(batch_acts,f)		
        		
		print(f"Batch {batch} finished:")
		for agent in env.agents:
			print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")		

Environment Toy-NECOM-Facultative created successfully!.
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


 Hold on, bringing the creitc network to range...

 Done!

Batch 0 finished:
agent1 return is:  -28.114913270986314
agent2 return is:  -8.442885776386422


 Hold on, bringing the creitc network to range...

 Done!

Batch 0 finished:
agent1 return is:  -28.114913270986314
agent2 return is:  -8.442885776386422
Batch 1 finished:
agent1 return is:  -27.88420291667081
agent2 return is:  -8.946670081882866
Batch 1 finished:
agent1 return is:  -27.88420291667081
agent2 return is:  -8.946670081882866
Batch 2 finished:
agent1 return is:  -26.36136426922207
agent2 return is:  -8.704309789626535
Batch 2 finished:
agent1 return is:  -26.36136426922207
agent2 return is:  -8.704309789626535
Batch 3 finished:
agent1 return is:  -26.124413740007213
agent2 return is:  -7.210674053515518
Batch 3 finished:
agent1 return is:  -26.124413740007213
agent2 return is:  -7.210674053515518
Batch 4 finished:
agent1 return is:  -25.098032069142185
agent2 return is:  -8.960677758995805
Batch 4 finished:
agent1 return is:  -25.098032069142185
agent2 return is:  -8.960677758995805
Batch 5 finished:
agent1 return is:  -25.859658208988343
agent2 return is:  -10.208980300111897
Batch 5 finished:
agent1 return is:  -25.859658208988

In [ ]:
model_base = cobra.io.read_sbml_model("iML1515.xml")
medium = model_base.medium.copy()
test_model = model_base.copy()

knockouts_gene_names = [
    "trpC",
    "pheA",
]

exchange_reactions = {
    "trpC": "EX_trp__L_e",
    "pheA": "EX_phe__L_e",
}

exchange_species = {}
exchange_mets = {}
for i in exchange_reactions.items():
    exchange_mets[i[0]] = list(test_model.reactions.get_by_id(i[1]).metabolites.keys())[
        0
    ].id

gene_ids = {}
for ko_gene in knockouts_gene_names:
    for gene in test_model.genes:
        if gene.name == ko_gene:
            gene_ids[ko_gene] = gene.id
            break

from itertools import combinations

knockouts = set()
for i in combinations(knockouts_gene_names, 2):
    if set(i) not in knockouts:
        knockouts.add(frozenset(i))

unique_knockouts = [tuple(i) for i in knockouts]

ic={
    key.lstrip("EX_"):10000 for key,val in model_base.medium.items() 
}

ic['glc__D_e']=100
ic['agent1']=0.05
ic['agent2']=0.05
for ko in unique_knockouts:
    model1 = model_base.copy()
    model2 = model_base.copy()
    model1.remove_reactions(model1.genes.get_by_id(gene_ids[ko[0]]).reactions)
    model2.remove_reactions(model2.genes.get_by_id(gene_ids[ko[1]]).reactions)
    model1.exchange_reactions = tuple([model1.reactions.index(i) for i in model1.exchanges])
    model2.exchange_reactions = tuple([model2.reactions.index(i) for i in model2.exchanges])
    model1.biomass_ind=model1.reactions.index("BIOMASS_Ec_iML1515_core_75p37M")
    model2.biomass_ind=model2.reactions.index("BIOMASS_Ec_iML1515_core_75p37M")
    model1.solver = "glpk"
    model2.solver = "glpk"
    if model1.optimize().objective_value != 0 or model2.optimize().objective_value != 0:
        rich.print(f"[yellow]Skipping {ko} because at least one organism can grow without auxotrophy")
        continue
    else:
        rich.print(f"[green]Non of the KOs can grow without auxotrophy: Running {ko}")
    ko_name = ko[0] + "_" + ko[1]
    agent1 = tk.Agent(
        "agent1",
        model=model1,
        actor_network=tk.NN,
        critic_network=tk.NN,
        clip=0.1,
        lr_actor=0.0001,
        lr_critic=0.001,
        grad_updates=10,
        optimizer_actor=torch.optim.Adam,
        optimizer_critic=torch.optim.Adam,
        observables=[
            "agent1",
            "agent2",
            "glc__D_e",
            exchange_mets[ko[0]],
            exchange_mets[ko[1]],
        ],
        actions=[exchange_reactions[ko[0]], exchange_reactions[ko[1]]],
        gamma=1,
    )
    agent2 = tk.Agent(
        "agent2",
        model=model2,
        actor_network=tk.NN,
        critic_network=tk.NN,
        clip=0.1,
        lr_actor=0.0001,
        lr_critic=0.001,
        grad_updates=10,
        optimizer_actor=torch.optim.Adam,
        optimizer_critic=torch.optim.Adam,
        observables=[
            "agent1",
            "agent2",
            "glc__D_e",
            exchange_mets[ko[0]],
            exchange_mets[ko[1]],
        ],
        actions=[exchange_reactions[ko[0]], exchange_reactions[ko[1]]],
        gamma=1,
    )

    env = tk.Environment(
        ko_name,
        agents=[agent1, agent2],
        dilution_rate=0,
        extracellular_reactions=[],
        initial_condition=ic,
        inlet_conditions={},
        max_c={},
        dt=0.1,
        episode_time=20,  ##TOBECHANGED
        number_of_batches=5000,  ##TOBECHANGED
        episodes_per_batch=1,
    )

    env.rewards = {agent.name: [] for agent in env.agents}

    if not os.path.exists(f"Results/{env.name}"):
        os.makedirs(f"Results/{env.name}")

for batch in range(env.number_of_batches):
	batch_obs,batch_acts, batch_log_probs, batch_rtgs=tk.rollout(env)
	for agent in env.agents:
		V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
		A_k = batch_rtgs[agent.name] - V.detach()   
		A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-5) 
		if batch==0:
			rich.print("[bold yellow] Hold on, bringing the creitc network to range...[/bold yellow]")
			err=21
			while err>20:
				V, _= agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step() 
				err=critic_loss.item()
			rich.print("[bold green] Done![/bold green]")
		else: 			
			for _ in range(agent.grad_updates):                                                      
				V, curr_log_probs = agent.evaluate(batch_obs[agent.name],batch_acts[agent.name])
				ratios = torch.exp(curr_log_probs - batch_log_probs[agent.name])
				surr1 = ratios * A_k.detach()
				surr2 = torch.clamp(ratios, 1 - agent.clip, 1 + agent.clip) * A_k
				actor_loss = (-torch.min(surr1, surr2)).mean()
				critic_loss = nn.MSELoss()(V, batch_rtgs[agent.name])
				agent.optimizer_policy_.zero_grad()
				actor_loss.backward(retain_graph=False)
				agent.optimizer_policy_.step()
				agent.optimizer_value_.zero_grad()
				critic_loss.backward()
				agent.optimizer_value_.step()                                                            	
	if batch%500==0:		
		with open(f"Results/{env.name}/{env.name}_{batch}.pkl", 'wb') as f:
			pickle.dump(env, f)
		with open(f"Results/{env.name}/observations_{batch}.pkl", 'wb') as f:
			pickle.dump(batch_obs,f)		
		with open(f"Results/{env.name}/actions_{batch}.pkl", 'wb') as f:
			pickle.dump(batch_acts,f)		
    		
	print(f"Batch {batch} finished:")
	for agent in env.agents:
		print(f"{agent.name} return is:  {np.mean(env.rewards[agent.name][-env.episodes_per_batch:])}")	

Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpf0y2ivtv.lp
Reading time = 0.02 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmp7wzh5z0a.lp
Reading time = 0.01 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Read LP format model from file /var/folders/jk/fr50qn391k794svhntbw333c0000gn/T/tmpsq04acmb.lp
Reading time = 0.02 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


Non of the KOs can grow without auxotrophy: Running ('trpC', 'pheA')

Environment trpC_pheA created successfully!.


 Hold on, bringing the creitc network to range...

 Done!

 Hold on, bringing the creitc network to range...

 Done!

Batch 0 finished:
agent1 return is:  -90.12790074852805
agent2 return is:  -106.0
Batch 1 finished:
agent1 return is:  -96.24234936460252
agent2 return is:  -105.60251919263362
